In [1]:
from modelUXception import *
from data_3ch import *

vid="v057_5" #version id should match the file number 
# The final number can be [1:5] and denotes the cross-validation order
# Ran DellWS with GeForce RTX3060 GPU

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=45,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

batch_size=3
learning_rate=1e-4
loss=dice_loss

train_gen = trainGenerator(batch_size,'afm_cell_512/train0'+vid[-1],'afmheideffri','labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'afm_cell_512/valid0'+vid[-1],'afmheideffri','labels',data_gen_args,save_to_dir = None)

# 26 images are used for training, 4 images for validating and 4 images for testing
train_steps = 24//batch_size
valid_steps = 6//batch_size

model = UXception()

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files/unet_cell_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=20*train_steps, validation_steps=valid_steps, 
                    epochs=100, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Epoch 1/100
160/160 [==============================] - ETA: 0s - loss: 0.2494 - acc: 0.9085 - dice_coef: 0.7506 - iou: 0.6280Found 6 images belonging to 1 classes.
Found 6 images belonging to 1 classes.
160/160 [==============================] - 107s 509ms/step - loss: 0.2494 - acc: 0.9085 - dice_coef: 0.7506 - iou: 0.6280 - val_loss: 0.3166 - val_acc: 0.8852 - val_dice_coef: 0.6834 - val_iou: 0.5214

Epoch 00001: saving model to files\unet_cell_seg_v057_5.hdf5


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
160/160 [==============================] - 79s 494ms/step - loss: 0.1135 - acc: 0.9664 - dice_coef: 0.8865 - iou: 0.8032 - val_loss: 0.1558 - val_acc: 0.9416 - val_dice_coef: 0.8442 - val_iou: 0.7310

Epoch 00002: saving model to files\unet_cell_seg_v057_5.hdf5
Epoch 3/100
160/160 [==============================] - 77s 481ms/step - loss: 0.0889 - acc: 0.9734 - dice_coef: 0.9111 - iou: 0.8411 - val_loss: 0.0818 - val_acc: 0.9726 - val_dice_coef: 0.9182 - val_iou: 0.8497

Epoch 00003: saving model to files\unet_cell_seg_v057_5.hdf5
Epoch 4/100
160/160 [==============================] - 80s 498ms/step - loss: 0.0826 - acc: 0.9760 - dice_coef: 0.9174 - iou: 0.8528 - val_loss: 0.0786 - val_acc: 0.9795 - val_dice_coef: 0.9214 - val_iou: 0.8545

Epoch 00004: saving model to files\unet_cell_seg_v057_5.hdf5
Epoch 5/100
160/160 [==============================] - 79s 496ms/step - loss: 0.0784 - acc: 0.9773 - dice_coef: 0.9216 - iou: 0.8605 - val_loss: 0.0604 - val_acc: 0.9751 - val_di

### validate your model and save predicted results

In [3]:
validGene = testGenerator("afm_cell_512/valid0"+vid[-1]+"/predhdf")
model.load_weights("files/unet_cell_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,6,verbose=1)
saveResult("afm_cell_512/valid0"+vid[-1]+"/predhdf",results,vid[:4])

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


6/6 [==============================] - 4s 110ms/step
